In [4]:
import numpy as np
from scipy.interpolate import interp1d as interp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from src.thunderflask import thunderflask
from src.bacteria import strain
from src.ffgen import ffgen
from src.codonTable import codonTable
from src.codonUtils import utils
from src.bacteria import strain
import pickle
from src.codonOptimizer import tableOptimizer
from queue import Queue as queue
from random import choice
from matplotlib.colors import LinearSegmentedColormap as LSC
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cProfile
import copy
import random

In [5]:
#CREATES TABLES FROM 8-20 AMINO ACIDS THAT PRIORITIZE MAXIMIZING DISTANCE 
rc = utils.standardTable
#Break table into dictionaries for each amino acid
Fblock = dict((key,value) for key, value in rc.items() if value == 'F')
Lblock = dict((key,value) for key, value in rc.items() if value == 'L')
Sblock = dict((key,value) for key, value in rc.items() if value == 'S')
Yblock = dict((key,value) for key, value in rc.items() if value == 'Y')
Cblock = dict((key,value) for key, value in rc.items() if value == 'C')
Wblock = dict((key,value) for key, value in rc.items() if value == 'W')
Pblock = dict((key,value) for key, value in rc.items() if value == 'P')
Hblock = dict((key,value) for key, value in rc.items() if value == 'H')
Qblock = dict((key,value) for key, value in rc.items() if value == 'Q')
Rblock = dict((key,value) for key, value in rc.items() if value == 'R')
Iblock = dict((key,value) for key, value in rc.items() if value == 'I')
Mblock = dict((key,value) for key, value in rc.items() if value == 'M')
Tblock = dict((key,value) for key, value in rc.items() if value == 'T')
Vblock = dict((key,value) for key, value in rc.items() if value == 'V')
Nblock = dict((key,value) for key, value in rc.items() if value == 'N')
Ablock = dict((key,value) for key, value in rc.items() if value == 'A')
Dblock = dict((key,value) for key, value in rc.items() if value == 'D')
Eblock = dict((key,value) for key, value in rc.items() if value == 'E')
Gblock = dict((key,value) for key, value in rc.items() if value == 'G')
Kblock = dict((key,value) for key, value in rc.items() if value == 'K')
stopBlock = dict((key,value) for key, value in rc.items() if value == '*')
blockArray = [Fblock, Lblock, Sblock, Yblock, Cblock, Wblock, Pblock, Hblock, Qblock, Rblock,
                Iblock, Mblock, Tblock, Vblock, Nblock, Ablock, Dblock, Eblock, Gblock, Kblock, stopBlock]
        
#Iterate through all amino acid blocks selecting one key and setting the rest map to stop
firstEight = ['UUU','UGG','CAG','CGU','AUG','ACU','GCG','GAU']
choices2 = []
for i in range (len(blockArray)):
    choices2.append(random.choice(list(blockArray[i])))
    for key in blockArray[i]: 
        if key not in firstEight:
            blockArray[i][key] = '*'
greenTable8 = {}
greenTable9 = {}
greenTable10 = {}
greenTable11 = {}
greenTable12 = {}
greenTable13 = {}
greenTable14 = {}            
greenTable15 = {}
greenTable16 = {}
greenTable17 = {}
greenTable18 = {}
greenTable19 = {}
greenTable20 = {}
for i in range (len(blockArray)):
    greenTable8.update(blockArray[i])
greenTable9.update(greenTable8)
greenTable9['AAA'] = 'K'
greenTable10.update(greenTable9)
greenTable10['GGA'] = 'G'
greenTable11.update(greenTable10)  
greenTable11['UCA'] = 'S'
greenTable12.update(greenTable11) 
greenTable12['CCC'] = 'P'
greenTable13.update(greenTable12) 
greenTable13['CUA'] ='L'
greenTable14.update(greenTable13)
greenTable14['UAC'] = 'Y'
greenTable15.update(greenTable14)
greenTable15['GUC'] = 'V'
greenTable16.update(greenTable15)
greenTable16['CAU'] = 'H'
greenTable17.update(greenTable16)  
greenTable17['GAG'] = 'E'
greenTable18.update(greenTable17) 
greenTable18['AUU'] = 'I'
greenTable19.update(greenTable18) 
greenTable19['AAC'] ='N'
greenTable20.update(greenTable19)
greenTable20['UGU'] = 'C'

In [6]:
codonTable(table=greenTable18).codonTable==codonTable(table=greenTable17).codonTable

U     C     A     G
U U   True  True  True  True
  C   True  True  True  True
  A   True  True  True  True
  G   True  True  True  True
C U   True  True  True  True
  C   True  True  True  True
  A   True  True  True  True
  G   True  True  True  True
A U  False  True  True  True
  C   True  True  True  True
  A   True  True  True  True
  G   True  True  True  True
G U   True  True  True  True
  C   True  True  True  True
  A   True  True  True  True
  G   True  True  True  True

In [7]:
#CREATES TABLES FROM 14-20 AMINO ACIDS THAT BALANCE MAXIMIZING DISTANCE AND BIOLOGICAL REALIZABILITY
rc = utils.standardTable
#Break table into dictionaries for each amino acid
Fblock = dict((key,value) for key, value in rc.items() if value == 'F')
Lblock = dict((key,value) for key, value in rc.items() if value == 'L')
Sblock = dict((key,value) for key, value in rc.items() if value == 'S')
Yblock = dict((key,value) for key, value in rc.items() if value == 'Y')
Cblock = dict((key,value) for key, value in rc.items() if value == 'C')
Wblock = dict((key,value) for key, value in rc.items() if value == 'W')
Pblock = dict((key,value) for key, value in rc.items() if value == 'P')
Hblock = dict((key,value) for key, value in rc.items() if value == 'H')
Qblock = dict((key,value) for key, value in rc.items() if value == 'Q')
Rblock = dict((key,value) for key, value in rc.items() if value == 'R')
Iblock = dict((key,value) for key, value in rc.items() if value == 'I')
Mblock = dict((key,value) for key, value in rc.items() if value == 'M')
Tblock = dict((key,value) for key, value in rc.items() if value == 'T')
Vblock = dict((key,value) for key, value in rc.items() if value == 'V')
Nblock = dict((key,value) for key, value in rc.items() if value == 'N')
Ablock = dict((key,value) for key, value in rc.items() if value == 'A')
Dblock = dict((key,value) for key, value in rc.items() if value == 'D')
Eblock = dict((key,value) for key, value in rc.items() if value == 'E')
Gblock = dict((key,value) for key, value in rc.items() if value == 'G')
Kblock = dict((key,value) for key, value in rc.items() if value == 'K')
stopBlock = dict((key,value) for key, value in rc.items() if value == '*')
blockArray = [Fblock, Lblock, Sblock, Yblock, Cblock, Wblock, Pblock, Hblock, Qblock, Rblock,
                Iblock, Mblock, Tblock, Vblock, Nblock, Ablock, Dblock, Eblock, Gblock, Kblock, stopBlock]
        
#Iterate through all amino acid blocks selecting one key and setting the rest map to stop
firstFourteen = ['UUU','UCA','UAC','CUG','CCC','CAU','CGA','AUC','ACG','AAA','GUA','GCU','GAG','GGC']
choices2 = []
for i in range (len(blockArray)):
    choices2.append(random.choice(list(blockArray[i])))
    for key in blockArray[i]: 
        if key not in firstFourteen:
            blockArray[i][key] = '*'           
balanced14 = {}
balanced15 = {}
balanced16 = {}
balanced17 = {}
balanced18 = {}
balanced19 = {}
balanced20 = {}
for i in range (len(blockArray)):
    balanced14.update(blockArray[i])
balanced15.update(balanced14)
balanced15['GAC'] = 'D'
balanced16.update(balanced15)
balanced16['CAG'] = 'Q'
balanced17.update(balanced16)  
balanced17['AAU'] = 'N'
balanced18.update(balanced17) 
balanced18['AUG'] = 'M'
balanced19.update(balanced18) 
balanced19['UGG'] ='W'
balanced20.update(balanced19)
balanced20['UGU'] = 'C'

In [8]:
#CREATES TABLES FROM 10-20 AMINO ACIDS THAT PRIORITIZE ABIOTIC AMINO ACIDS AND BIOLOGICAL FEASIBILITY 
rc = utils.standardTable
#Break table into dictionaries for each amino acid
Fblock = dict((key,value) for key, value in rc.items() if value == 'F')
Lblock = dict((key,value) for key, value in rc.items() if value == 'L')
Sblock = dict((key,value) for key, value in rc.items() if value == 'S')
Yblock = dict((key,value) for key, value in rc.items() if value == 'Y')
Cblock = dict((key,value) for key, value in rc.items() if value == 'C')
Wblock = dict((key,value) for key, value in rc.items() if value == 'W')
Pblock = dict((key,value) for key, value in rc.items() if value == 'P')
Hblock = dict((key,value) for key, value in rc.items() if value == 'H')
Qblock = dict((key,value) for key, value in rc.items() if value == 'Q')
Rblock = dict((key,value) for key, value in rc.items() if value == 'R')
Iblock = dict((key,value) for key, value in rc.items() if value == 'I')
Mblock = dict((key,value) for key, value in rc.items() if value == 'M')
Tblock = dict((key,value) for key, value in rc.items() if value == 'T')
Vblock = dict((key,value) for key, value in rc.items() if value == 'V')
Nblock = dict((key,value) for key, value in rc.items() if value == 'N')
Ablock = dict((key,value) for key, value in rc.items() if value == 'A')
Dblock = dict((key,value) for key, value in rc.items() if value == 'D')
Eblock = dict((key,value) for key, value in rc.items() if value == 'E')
Gblock = dict((key,value) for key, value in rc.items() if value == 'G')
Kblock = dict((key,value) for key, value in rc.items() if value == 'K')
stopBlock = dict((key,value) for key, value in rc.items() if value == '*')
blockArray = [Fblock, Lblock, Sblock, Yblock, Cblock, Wblock, Pblock, Hblock, Qblock, Rblock,
                Iblock, Mblock, Tblock, Vblock, Nblock, Ablock, Dblock, Eblock, Gblock, Kblock, stopBlock]
        
#Iterate through all amino acid blocks selecting one key and setting the rest map to stop
firstTen = ['UCA','CUG','CCU','AUU','ACC','GUC','GCG','GAU','GAG','GGA']
choices2 = []
for i in range (len(blockArray)):
    choices2.append(random.choice(list(blockArray[i])))
    for key in blockArray[i]: 
        if key not in firstTen:
            blockArray[i][key] = '*'
abiotic10 = {}
abiotic11 = {}
abiotic12 = {}
abiotic13 = {}
abiotic14 = {}            
abiotic15 = {}
abiotic16 = {}
abiotic17 = {}
abiotic18 = {}
abiotic19 = {}
abiotic20 = {}
for i in range (len(blockArray)):
    abiotic10.update(blockArray[i])
abiotic11.update(abiotic10)  
abiotic11['CGC'] = 'R'
abiotic12.update(abiotic11) 
abiotic12['CAA'] = 'Q'
abiotic13.update(abiotic12) 
abiotic13['AAA'] ='K'
abiotic14.update(abiotic13)
abiotic14['CAC'] = 'H'
abiotic15.update(abiotic14)
abiotic15['UUU'] = 'F'
abiotic16.update(abiotic15)
abiotic16['UAC'] = 'Y'
abiotic17.update(abiotic16)  
abiotic17['AAU'] = 'N'
abiotic18.update(abiotic17) 
abiotic18['AUG'] = 'M'
abiotic19.update(abiotic18) 
abiotic19['UGG'] ='W'
abiotic20.update(abiotic19)
abiotic20['UGU'] = 'C'

In [9]:
#############################################################################
# SECTION BELOW BUILDS TABLES UP TO 20 AAs FOR GREEN (DISTANCE MAXIMIZING), #
# ABIOTIC (BIOLOGICAL RELEVANCE MAXIMIZING), AND BALANCED                   #
# (BALANCES MAXIMIZING DISTANCE AND BIOLOGICAL REALIZABILITY)               #
#############################################################################

In [10]:
green20Table = codonTable(greenTable20)
green19Table = codonTable(greenTable19)
green18Table = codonTable(greenTable18)
green17Table = codonTable(greenTable17)
green16Table = codonTable(greenTable16)
green15Table = codonTable(greenTable15)
green14Table = codonTable(greenTable14)
green13Table = codonTable(greenTable13)
green12Table = codonTable(greenTable12)
green11Table = codonTable(greenTable11)
green10Table = codonTable(greenTable10)
green9Table = codonTable(greenTable9)
green8Table = codonTable(greenTable8)
balanced20Table = codonTable(balanced20)
balanced19Table = codonTable(balanced19)
balanced18Table = codonTable(balanced18)
balanced17Table = codonTable(balanced17)
balanced16Table = codonTable(balanced16)
balanced15Table = codonTable(balanced15)
balanced14Table = codonTable(balanced14)
abiotic20Table = codonTable(abiotic20)
abiotic19Table = codonTable(abiotic19)
abiotic18Table = codonTable(abiotic18)
abiotic17Table = codonTable(abiotic17)
abiotic16Table = codonTable(abiotic16)
abiotic15Table = codonTable(abiotic15)
abiotic14Table = codonTable(abiotic14)
abiotic13Table = codonTable(abiotic13)
abiotic12Table = codonTable(abiotic12)
abiotic11Table = codonTable(abiotic11)
abiotic10Table = codonTable(abiotic10)

In [11]:
#TABLE CHECKS ARE BELOW

In [12]:
abiotic10Table.dictToTable()

U        C        A        G
U U  UUU : *  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : *  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : *
C U  CUU : *  CCU : P  CAU : *  CGU : *
  C  CUC : *  CCC : *  CAC : *  CGC : *
  A  CUA : *  CCA : *  CAA : *  CGA : *
  G  CUG : L  CCG : *  CAG : *  CGG : *
A U  AUU : I  ACU : *  AAU : *  AGU : *
  C  AUC : *  ACC : T  AAC : *  AGC : *
  A  AUA : *  ACA : *  AAA : *  AGA : *
  G  AUG : *  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : D  GGU : *
  C  GUC : V  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : *  GAA : *  GGA : G
  G  GUG : *  GCG : A  GAG : E  GGG : *

In [13]:
abiotic20Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : C
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : P  CAU : *  CGU : *
  C  CUC : *  CCC : *  CAC : H  CGC : R
  A  CUA : *  CCA : *  CAA : Q  CGA : *
  G  CUG : L  CCG : *  CAG : *  CGG : *
A U  AUU : I  ACU : *  AAU : N  AGU : *
  C  AUC : *  ACC : T  AAC : *  AGC : *
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : D  GGU : *
  C  GUC : V  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : *  GAA : *  GGA : G
  G  GUG : *  GCG : A  GAG : E  GGG : *

In [14]:
green15Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : *  CGU : R
  C  CUC : *  CCC : P  CAC : *  CGC : *
  A  CUA : L  CCA : *  CAA : *  CGA : *
  G  CUG : *  CCG : *  CAG : Q  CGG : *
A U  AUU : *  ACU : T  AAU : *  AGU : *
  C  AUC : *  ACC : *  AAC : *  AGC : *
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : D  GGU : *
  C  GUC : V  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : *  GAA : *  GGA : G
  G  GUG : *  GCG : A  GAG : *  GGG : *

In [15]:
green20Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : C
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : H  CGU : R
  C  CUC : *  CCC : P  CAC : *  CGC : *
  A  CUA : L  CCA : *  CAA : *  CGA : *
  G  CUG : *  CCG : *  CAG : Q  CGG : *
A U  AUU : I  ACU : T  AAU : *  AGU : *
  C  AUC : *  ACC : *  AAC : N  AGC : *
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : D  GGU : *
  C  GUC : V  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : *  GAA : *  GGA : G
  G  GUG : *  GCG : A  GAG : E  GGG : *

In [16]:
balanced14Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : *
C U  CUU : *  CCU : *  CAU : H  CGU : *
  C  CUC : *  CCC : P  CAC : *  CGC : *
  A  CUA : *  CCA : *  CAA : *  CGA : R
  G  CUG : L  CCG : *  CAG : *  CGG : *
A U  AUU : *  ACU : *  AAU : *  AGU : *
  C  AUC : I  ACC : *  AAC : *  AGC : *
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : *  ACG : T  AAG : *  AGG : *
G U  GUU : *  GCU : A  GAU : *  GGU : *
  C  GUC : *  GCC : *  GAC : *  GGC : G
  A  GUA : V  GCA : *  GAA : *  GGA : *
  G  GUG : *  GCG : *  GAG : E  GGG : *

In [22]:
balanced14Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : *
C U  CUU : *  CCU : *  CAU : H  CGU : *
  C  CUC : *  CCC : P  CAC : *  CGC : *
  A  CUA : *  CCA : *  CAA : *  CGA : R
  G  CUG : L  CCG : *  CAG : *  CGG : *
A U  AUU : *  ACU : *  AAU : *  AGU : *
  C  AUC : I  ACC : *  AAC : *  AGC : *
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : *  ACG : T  AAG : *  AGG : *
G U  GUU : *  GCU : A  GAU : *  GGU : *
  C  GUC : *  GCC : *  GAC : *  GGC : G
  A  GUA : V  GCA : *  GAA : *  GGA : *
  G  GUG : *  GCG : *  GAG : E  GGG : *

In [23]:
path = '/home/jonathan/Lab/Fast Fail/res/Reductionist Code/'
with open(path+'reductionist20.pickle', 'wb') as handle:
    pickle.dump(green20Table.codonDict, handle)
with open(path+'reductionist14.pickle', 'wb') as handle:
    pickle.dump(balanced14Table.codonDict, handle)